# AWBS - Project 2

author: Mateusz Domalewski

group: 2

index: 288761

In [2]:
### importing libraries
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

In [4]:
### importing data sources
E_COMMERCE = './data/E-commerce_Customer_Behavior_Sheet1.csv'

## satisfaction level of customers of e-commerce shop

In [6]:
ecomm = pl.read_csv(E_COMMERCE)
ecomm.describe()

statistic,Customer ID,Gender,Age,City,Membership Type,Total Spend,Items Purchased,Average Rating,Discount Applied,Days Since Last Purchase,Satisfaction Level
str,f64,str,f64,str,str,f64,f64,f64,f64,f64,str
"""count""",350.0,"""350""",350.0,"""350""","""350""",350.0,350.0,350.0,350.0,350.0,"""348"""
"""null_count""",0.0,"""0""",0.0,"""0""","""0""",0.0,0.0,0.0,0.0,0.0,"""2"""
"""mean""",275.5,null,33.597143,null,null,845.381714,12.6,4.019143,0.5,26.588571,null
"""std""",101.180532,null,4.870882,null,null,362.058695,4.155984,0.580539,null,13.440813,null
"""min""",101.0,"""Female""",26.0,"""Chicago""","""Bronze""",410.8,7.0,3.0,0.0,9.0,"""Neutral"""
"""25%""",188.0,null,30.0,null,null,500.75,9.0,3.5,null,15.0,null
"""50%""",276.0,null,33.0,null,null,780.2,12.0,4.1,null,23.0,null
"""75%""",363.0,null,37.0,null,null,1160.6,15.0,4.5,null,38.0,null
"""max""",450.0,"""Male""",43.0,"""San Francisco""","""Silver""",1520.1,21.0,4.9,1.0,63.0,"""Unsatisfied"""


In [9]:
## drop nulls for classes
ecomm = ecomm.drop_nulls()

### assessment of the dependence of customer

### satisfaction level on other parameters